# ADS Coursework 2023 - Geographic Sentiment Distribution of Trending Social Media Topics 

## Initial Setup - Twitter

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# query = "(ai OR chat-gpt) -ad -free -purchase -premium -avail -claim -giveaway -participants -Telegram -winner -win -credits -token -tokens -football -aiART -artwork -art -cosplay -character -waifu -generated min_faves:1000 lang:en"
query = '(ai OR "artificial intelligence") -filter:links -ad -free -purchase -premium -avail -claim -giveaway -participants -Telegram -winner -win -credits -token -tokens -aiART -artwork -art -cosplay -character -waifu -generated lang:en'
# -ad -free -purchase -premium -avail -claim -giveaway -participants -Telegram -winner -win -credits -token -tokens -football -aiART -artwork -art -cosplay -character lang:en" 
# -waifu -generated min_faves:1000 lang:en"
tweets = []
limit = 50


for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.content])
        
df = pd.DataFrame(tweets, columns=['Date', 'Tweet'])

In [ ]:
print(df.to_string())

In [17]:
import re
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from geopy.geocoders import Nominatim
from functools import partial

keywords = "(AI, Artificial Intelligence, artificial intelligence, ai) -filter:links -ad -free -purchase -premium -avail -claim -giveaway -participants -Telegram -winner -win -credits -token -tokens -football -aiART -artwork -art -cosplay -character -waifu -generated lang:en"
start_date = dt.datetime(2017, 1, 1) # Start date
end_date = dt.datetime(2019, 12, 31) # End date
limit = 10 # Number of tweets per month

queries = []
while start_date < end_date:
    month_start = start_date
    month_end = start_date + relativedelta(months=+1) - timedelta(days=1)
    if month_end > end_date:
        month_end = end_date
    
    query = f'{keywords} until:{month_end.date()} since:{month_start.date()}'
    queries.append(query)
    
    start_date = month_end + timedelta(days=1)

In [32]:
pattern = r"^\w+,\s\w+$"

tweets = []
tweets_content = []
count = 0
for query in tqdm(queries): 
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if  count == limit:
            count = 0
            break
        if  tweet.user.location != '':
            # Use regular expression to match the location pattern
            match = re.match(pattern, tweet.user.location)
            if match:
                tweet.user.location = match.group()
                tweets.append([tweet.date, tweet.user.username, tweet.content, tweet.user.location])
                count += 1

    columns = ['Date', 'User', 'Content', 'Location']

df = pd.DataFrame(tweets, columns=columns)
df

  0%|                                                                                           | 0/36 [00:00<?, ?it/s]C:\Users\arafa\AppData\Local\Temp\ipykernel_13172\1957173761.py:17: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.user.username, tweet.content, tweet.user.location])
100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [01:58<00:00,  3.30s/it]


,Date,User,Content,Location
0,2017-01-30 22:46:00+00:00,RandomizerTest,"""Made with Artificial Intelligence, I call it ...","Lakewood, OH"
1,2017-01-30 22:46:00+00:00,RandomizerTest,"""The combination of human and artificial intel...","Lakewood, OH"
2,2017-01-30 22:45:59+00:00,RandomizerTest,"""An artificial intelligence (AI) agent based o...","Lakewood, OH"
3,2017-01-30 21:52:40+00:00,RandomizerTest,"""What if the US was run by some kind of artifi...","Lakewood, OH"
4,2017-01-30 20:57:10+00:00,RandomizerTest,"""Learning about Machine Learning/Artificial In...","Lakewood, OH"
...,...,...,...,...
355,2019-12-30 13:00:15+00:00,QContrarian,"@scheplick Remember, artificial intelligence i...","Washington, DC"
356,2019-12-30 12:00:00+00:00,getppcexpo,"As artificial intelligence (AI) gets smarter, ...","Houston, Texas"
357,2019-12-30 02:52:10+00:00,wfryer,"@grendelrick A10 In addition, artificial intel...","Charlotte, NC"
358,2019-12-29 19:15:04+00:00,ivanatilca,🤯 QUESTION: Are we in a historical moment in w...,"Brooklyn, NY"


In [33]:
# to save to csv
df.to_csv('tweets.csv')

In [34]:
geolocator = Nominatim(user_agent="coursework2023Notebook")
for location in df.Location:
    geocode = partial(geolocator.geocode, language="en")
    address = geocode(location)
    print(address)

Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Montclair, Essex County, New Jersey, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Wayne County, Pennsylvania, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, United States
Lakewood, Cuyahoga County, Ohio, 44107, Uni

Special Region of Yogyakarta, Indonesia
Wakiso, Central Region, Uganda
Washington, District of Columbia, United States
Charlottesville, Virginia, United States
Bradford, West Yorkshire, England, United Kingdom
Washington, District of Columbia, United States
Waunakee, Dane County, Wisconsin, 53597, United States
Boston, Suffolk County, Massachusetts, United States
Conduit Lane East, Hoddesdon, Broxbourne, Hertfordshire, England, EN11 8EA, United Kingdom
Gurgaon, Gurugram, Haryana, India
Brisbane City, Queensland, Australia
Arlington County, Virginia, United States
Bengaluru, Bangalore North, Bangalore Urban, Karnataka, India
Amarillo, Potter County, Texas, United States
Beijing, Dongcheng District, 100010, China
Lagos, Lagos Island, Lagos State, 100242, Nigeria
City of Edinburgh, Scotland, United Kingdom
Tver, Tver Oblast, Central Federal District, Russia
Berlin, Germany
Munich, Bavaria, Germany
Newcastle upon Tyne, North of Tyne, England, United Kingdom
Charlotte, Mecklenburg County, N

Washington, District of Columbia, United States
Gainesville, Alachua County, Florida, United States
Minneapolis, Hennepin County, Minnesota, United States
Nairobi, Nairobi County, Kenya
Lagos, Lagos Island, Lagos State, 100242, Nigeria
Liverpool, Liverpool City Region, England, United Kingdom
Bengaluru, Bangalore North, Bangalore Urban, Karnataka, India
Seattle, King County, Washington, United States
Blacksburg, Montgomery County, Virginia, United States
Vancouver, Metro Vancouver Regional District, British Columbia, Canada
Istanbul, Fatih, Istanbul, Marmara Region, 34126, Turkey
Padua, Padova, Veneto, Italy
Ibadan, Ibadan South West, Oyo State, 234102, Nigeria
Beijing, Dongcheng District, 100010, China
Oxford, Oxfordshire, England, United Kingdom
Rome, Roma Capitale, Lazio, Italy
Manhattan, New York County, New York, United States
Manhattan, New York County, New York, United States
Boston, Suffolk County, Massachusetts, United States
Awka, Awka South, Anambra State, Nigeria
Newport, N

In [4]:
# load model and tokeniser
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)
labels = ['Negative', 'Neutral', 'Positive']

In [14]:
highest_labels = []

for tweet in tweets_content:

    # preprocess tweet
    tweet_words = []

    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'

        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)

    # sentiment analysis
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    # output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    highest_label = 'None'
    highest_score = 0
    
    print(tweet)

    for i in range(len(scores)):

        l = labels[i]
        s = scores[i]
        if s > highest_score:
            highest_label = l
            highest_score = s
        print(l, s)
        
    highest_labels.append(highest_label)
    
    print('\n')

@JudgeCarter There have been tremendous advancement in AI (Artificial Intelligence), robotics and autonomous vehicles which is causing a great deal of concern for the people I talk to. They are afraid they will lose their jobs &amp; their children &amp;grandchildren won't have any how will they live?
Negative 0.46596152
Neutral 0.407507
Positive 0.12653144


@AhmadGanga The rise of Artificial intelligence(AI) is a big threat to humanity. We gonna lose our concept of humanity to algorithms.
Negative 0.7944987
Neutral 0.18753004
Positive 0.01797127


I was born and raised in #Germany. This country and its people always fascinate me in a certain way. When it comes down to it, they can function efficiently like a sophisticated machine.  So efficiently that even Artificial Intelligence is scared shitless.

#AI #CoronaCrisis
Negative 0.54484105
Neutral 0.30624062
Positive 0.14891835


@BillGates I disrespectfully decline your toxic vaccines. They haven't been tested, what are the ingredients

Artificial intelligence presents the most potent and holistic solution class to respond to this pressurized investment and trading environment. 

#investment #trading #opportunities #market #investments #AI
Negative 0.00555405
Neutral 0.14019471
Positive 0.85425127


Artificial Intelligence and Agriculture

Transformative companies are innovating agriculture with AI

Read more in the September 2022 issue of the Precision Agriculture Digital Digest to learn about the innovative companies ⤵️

#PrecisionAg #AgTech #PrecisionFarming #AI #OntAg
Negative 0.007375832
Neutral 0.51804703
Positive 0.47457713


I told my friend that I wanted to learn digital marketing first, then learn full-stack web development, before going into Data Science and Artificial Intelligence (AI). Lol!  Do you know that ‘who is this guy' kind of look? That's what I got.
Negative 0.101339415
Neutral 0.57360405
Positive 0.32505658


"The true challenge to artificial intelligence proved to be solving the tasks that are 

In [8]:
print(highest_labels)

['Negative', 'Negative', 'Negative', 'Negative', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Positive', 'Neutral', 'Negative', 'Positive', 'Positive', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Negative', 'Positive', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral']


## Initial Setup - Facebook

In [ ]:
import facebook

# create a GraphAPI object with your access token
graph = facebook.GraphAPI(access_token='EAAN22gDkdyQBAE9eZAETeTsvtQn3AC9t8UDylznGAF24ZBReqz6kzSNCMT2kKZBwfPZCgwUUKYBENrTFpZASuBdwzVpZBoF8KmCsH6ws8ZBcZAn7tnc7VGKPEWALtjCW8QpNBDZA4eVx1Ll5HOlimuQBhMAlSCMwWoBdzw0V58SbtMD7KqRpKFX2SVZCFYRIRpLjkDLPz0ApZCHmNNKgvePAnZAs5POio2F4HP2NQzmqYI3tnFv3VsZBgTbZBTRScmnVschfoZD', version="3.1")

# make a request to the search endpoint with query parameters
results = graph.request('bbc')

In [ ]:
# iterate through the search results
for post in results['data']:
    comment_count = post['comments']['summary']['total_count']
    if comment_count > 500:
        print(f"Post ID: {post['id']}")
        print(f"Message: {post['message']}")
        print(f"Comment count: {comment_count}")

## Initial Setup - Facebook 2

In [ ]:
import sys

In [ ]:
from facebook_scraper import get_posts

posts_for_analysis = []

for post in get_posts("698593531630485", cookies="from_browser", pages=100):
    if post["comments"] > 200:
        print(post["text"][:50])

In [ ]:
from facebook_scraper import get_posts

posts_for_analysis = []

for post in get_posts("Reuters", cookies=sys.path.append("./cookies"), pages=100, options={"posts_per_page": 10}):
    if post["comments"] > 200:
        print(post["text"][:50])